In [1]:
using Gmsh

gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 1)
gmsh.model.add("CircularCurve")

In [2]:
#Specimen
Length = 300
Width = 144
# Define unit cell count
nx = 12
# Parameters
r = 10
lc = 0.5
Theta_deg = 30.0

30.0

In [3]:
Theta = Theta_deg*pi/180
L = Width/((nx-1)*2*cos(Theta))
ny = ceil(Int, 1 + (Length / (2 * L * (1 + sin(Theta)))))

println(L)
println(ny)

t = L / r

7.5580398875732815
15


0.7558039887573281

In [4]:
# Rectangle at center
x0 = -t * 0.5
y0 = -L * 0.5
leg1 = gmsh.model.occ.addRectangle(x0, y0, 0.0, t, L)

# Outer arc sector
r1 = L + t * 0.5
cx = L * cos(Theta)
cy = 0.0

0.0

In [5]:
p1 = gmsh.model.occ.addPoint(cx, cy, 0, lc)
p2 = gmsh.model.occ.addPoint(cx - r1 * cos(Theta), r1 * sin(Theta), 0, lc)
p3 = gmsh.model.occ.addPoint(cx, r1, 0, lc)

arc1 = gmsh.model.occ.addCircleArc(p2, p1, p3)
l1 = gmsh.model.occ.addLine(p3, p1)
l2 = gmsh.model.occ.addLine(p1, p2)
loop1 = gmsh.model.occ.addCurveLoop([arc1, l1, l2])
surf1 = gmsh.model.occ.addPlaneSurface([loop1])


r2 = L - t * 0.5
p4 = gmsh.model.occ.addPoint(cx, cy, 0, lc)
p5 = gmsh.model.occ.addPoint(cx - r2 * cos(Theta), r2 * sin(Theta), 0, lc)
p6 = gmsh.model.occ.addPoint(cx, r2, 0, lc)

arc2 = gmsh.model.occ.addCircleArc(p5, p4, p6)
l3 = gmsh.model.occ.addLine(p6, p4)
l4 = gmsh.model.occ.addLine(p4, p5)

10

In [6]:
loop2 = gmsh.model.occ.addCurveLoop([arc2, l3, l4])
surf2 = gmsh.model.occ.addPlaneSurface([loop2])

gmsh.model.occ.synchronize()

cut_out = gmsh.model.occ.cut([(2, surf1)], [(2, surf2)])
gmsh.model.occ.synchronize()

leg2 = cut_out[1]

leg3 = gmsh.model.occ.copy(leg2)

gmsh.model.occ.symmetrize(leg3, 1.0, 0.0, 0.0, 0.0) 
gmsh.model.occ.synchronize()

leg4 = gmsh.model.occ.copy(leg2)
gmsh.model.occ.translate(leg4, -L*cos(Theta), -1.5*L, 0)

leg5 = gmsh.model.occ.copy(leg3)
gmsh.model.occ.translate(leg5, L*cos(Theta), -1.5*L, 0)
gmsh.model.occ.synchronize()

fused1, _ = gmsh.model.occ.fuse([(2, leg1)], leg2)
gmsh.model.occ.synchronize()

fused2, _ = gmsh.model.occ.fuse(fused1, leg3)
gmsh.model.occ.synchronize()

fused3, _ = gmsh.model.occ.fuse(fused2, leg4)
gmsh.model.occ.synchronize()

final_fused, _ = gmsh.model.occ.fuse(fused3, leg5)

gmsh.model.occ.synchronize()

In [7]:
# ------------------------------
# Unit cell box vectors
# ------------------------------
e1x = L * cos(Theta)
e1y = 0
e2x = -L * cos(Theta)
e2y = 0

org1 = -L-L*sin(Theta)
org2 = L+L*sin(Theta)

pA = gmsh.model.occ.addPoint(e1x,  e1y, 0, lc)
pB = gmsh.model.occ.addPoint(e2x,  e2y, 0, lc)
pC = gmsh.model.occ.addPoint(0,  org1, 0, lc)
pD = gmsh.model.occ.addPoint(0,  org2, 0, lc)

lA = gmsh.model.occ.addLine(pA, pC)
lB = gmsh.model.occ.addLine(pB, pC)
lC = gmsh.model.occ.addLine(pA, pD)
lD = gmsh.model.occ.addLine(pB, pD)

loopUC = gmsh.model.occ.addCurveLoop([lA, lB, lC, lD])
UC_box = gmsh.model.occ.addPlaneSurface([loopUC])

gmsh.option.setNumber("Geometry.ToleranceBoolean", 1e-6)
intersection, _ = gmsh.model.occ.intersect(final_fused, [(2, UC_box)])

Copy1 = gmsh.model.occ.copy(intersection)
gmsh.model.occ.translate(Copy1, L*cos(Theta), L+(L*sin(Theta)), 0)

final_fused2, _ = gmsh.model.occ.fuse(intersection, Copy1)

dx = 2 * L * cos(Theta)
dy = 2 * L + 2 * L * sin(Theta)
# ------------------------------
# Unit cell
# ------------------------------
UC_surfs = [(dim, tag) for (dim, tag) in final_fused2 if dim == 2]

Info    : Cannot bind existing OpenCASCADE surface 6 to second tag 10                                                                  
Info    : Could not preserve tag of 2D object 10 (->6)


5-element Vector{Tuple{Int32, Int32}}:
 (2, 1)
 (2, 6)
 (2, 7)
 (2, 8)
 (2, 9)

all_cells = []
# ------------------------------
# Array of Lattice
# ------------------------------
for j in 0:ny-1
    for i in 0:nx-1
        # Copy UC surfaces
        new_UC = gmsh.model.occ.copy(UC_surfs)
        # Translate by (i*dx, j*dy)
        gmsh.model.occ.translate(new_UC, i * dx, j * dy, 0.0)
        # Store the new surfaces
        append!(all_cells, new_UC)
    end
end

gmsh.model.occ.synchronize()


unique_cells = unique(all_cells)

gmsh.option.setNumber("Geometry.ToleranceBoolean", 1e-6)
intersection1, _ = gmsh.model.occ.intersect(final_fused, [(2, UC_box)])

x_rect = 0 
y_rect = 0
w_rect = Width   
h_rect = Length 

cover_rect = gmsh.model.occ.addRectangle(x_rect, y_rect, 0.0, w_rect, h_rect)

intersection_result, _ = gmsh.model.occ.intersect([(2, cover_rect)], unique_cells)
frag_surfs = [(dim, tag) for (dim, tag) in intersection_result if dim == 2]

cover_rect2 = gmsh.model.occ.addRectangle(x_rect, y_rect, 0.0, w_rect, h_rect)
fragment_result, _ = gmsh.model.occ.fragment([(2, cover_rect2)], intersection_result)
gmsh.model.occ.synchronize()
frag_surfs = [(dim, tag) for (dim, tag) in fragment_result if dim == 2]

lattice_tags = Set(tag for (dim, tag) in intersection_result if dim == 2)
lattice_surfs = [(dim, tag) for (dim, tag) in frag_surfs if tag in lattice_tags]
base_surfs = [(dim, tag) for (dim, tag) in frag_surfs if !(tag in lattice_tags)]

# ------------------------------
# Physical Group
# ------------------------------
if !isempty(lattice_surfs)
    pg_lattice = gmsh.model.addPhysicalGroup(2, [tag for (dim, tag) in lattice_surfs])
    gmsh.model.setPhysicalName(2, pg_lattice, "Lattice")
end

if !isempty(base_surfs)
    pg_base = gmsh.model.addPhysicalGroup(2, [tag for (dim, tag) in base_surfs])
    gmsh.model.setPhysicalName(2, pg_base, "Base")
end

# ------------------------------
# Define Top and Bottom Edges as Physical Groups
# ------------------------------

bp1 = gmsh.model.occ.addPoint(0, 0, 0, lc)
bp2 = gmsh.model.occ.addPoint(Width,  0, 0, lc)
tp1 = gmsh.model.occ.addPoint(0,  Length, 0, lc)
tp2 = gmsh.model.occ.addPoint(Width,  Length, 0, lc)

bottom_edge = gmsh.model.occ.addLine(bp1, bp2)
top_edge = gmsh.model.occ.addLine(tp1, tp2)
gmsh.model.occ.synchronize()

tag1 = gmsh.model.addPhysicalGroup(1, [bottom_edge], 10001)
tag2 = gmsh.model.addPhysicalGroup(1, [top_edge], 10002)

gmsh.model.setPhysicalName(1, tag1, "BottomEdge")
gmsh.model.setPhysicalName(1, tag2, "TopEdge")

for (dim, tag) in gmsh.model.getPhysicalGroups()
    entities = gmsh.model.getEntitiesForPhysicalGroup(dim, tag)
    if isempty(entities)
        println("Warning: Physical group with tag=$tag and dim=$dim is empty!")
    end
end

In [8]:
gmsh.option.setNumber("Mesh.MeshSizeMin", t/2)
gmsh.option.setNumber("Mesh.MeshSizeMax", t)

# Mesh and save
gmsh.model.mesh.generate(2)
gmsh.write("CircularCurve.msh")

println("Physical groups defined:")
for (dim, tag) in gmsh.model.getPhysicalGroups()
    name = gmsh.model.getPhysicalName(dim, tag)
    println(" - [$name] dim=$dim, tag=$tag")
end
gmsh.finalize()

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 14 (TrimmedCurve)
Info    : [ 10%] Meshing curve 20 (TrimmedCurve)
Info    : [ 20%] Meshing curve 25 (Line)
Info    : [ 20%] Meshing curve 26 (Line)
Info    : [ 30%] Meshing curve 30 (TrimmedCurve)
Info    : [ 30%] Meshing curve 31 (TrimmedCurve)
Info    : [ 40%] Meshing curve 32 (TrimmedCurve)
Info    : [ 40%] Meshing curve 33 (Circle)
Info    : [ 50%] Meshing curve 34 (Line)
Info    : [ 50%] Meshing curve 35 (Circle)
Info    : [ 60%] Meshing curve 39 (Line)
Info    : [ 60%] Meshing curve 40 (Line)
Info    : [ 70%] Meshing curve 41 (TrimmedCurve)
Info    : [ 70%] Meshing curve 42 (Line)
Info    : [ 80%] Meshing curve 43 (Line)
Info    : [ 80%] Meshing curve 44 (Circle)
Info    : [ 90%] Meshing curve 45 (Line)
Info    : [ 90%] Meshing curve 46 (Circle)
Info    : [100%] Meshing curve 47 (TrimmedCurve)
Info    : [100%] Meshing curve 48 (TrimmedCurve)
Info    : Done meshing 1D (Wall 0.00686979s, CPU 0.006829s)
Info    : Meshing 2D...


In [13]:
using GridapGmsh
model = GmshDiscreteModel("CircularCurve.msh")

Info    : Reading 'CircularCurve.msh'...
Info    : 41 entities
Info    : 183 nodes
Info    : 390 elements
Info    : Done reading 'CircularCurve.msh'


UnstructuredDiscreteModel()

In [18]:
using Gridap
writevtk(model,"model")

3-element Vector{Vector{String}}:
 ["model_0.vtu"]
 ["model_1.vtu"]
 ["model_2.vtu"]